<a href="https://colab.research.google.com/github/mileslilly2/code_for_whatever_you_are/blob/main/extract_text_from_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install PyPDF2

from google.colab import files
uploaded = files.upload()          # choose e.g.  "my_book.pdf"
pdf_name = next(iter(uploaded))    # grabs the uploaded filename


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


Saving Real-World Bug Hunting - A Field Guide to Web Hacking by Peter Yaworski.pdf to Real-World Bug Hunting - A Field Guide to Web Hacking by Peter Yaworski.pdf


In [ ]:
# --- Split the whole PDF text on "chapter", keep each chunk’s first 100 chars,
# --- then split that snippet on ":" and take the last piece.
#     Finally, print the two resulting lists.

import re
from PyPDF2 import PdfReader

# 1) load the PDF you uploaded in Cell 1
reader     = PdfReader(pdf_name)               # pdf_name came from files.upload()
full_text  = "\n".join(page.extract_text() or "" for page in reader.pages)

# 2) split on the word "chapter" (case-insensitive)
chunks = re.split(r'chapter', full_text, flags=re.I)[1:]   # skip any front-matter

# 3) grab first 100 characters of each chunk
first_100 = [chunk[:100] for chunk in chunks]

# 4) for each 100-char snippet, split on ":" and keep the last element
after_colon = [snippet.split(':')[-1].strip() for snippet in first_100]

# 5) print the lists
print("First-100-char snippets:\n", first_100, "\n")
print("Portion after last ':' in each snippet:\n", after_colon)


First-100-char snippets:
 [' 1: Bug Bounty Basics\n', ' 2: Open Redirect\n', ' 3: HTTP Parameter Pollution\n', ' 4: Cross-Site Request Forgery\n', ' 5: HTML Injection and Content Spooﬁng\n', ' 6: Carriage Return Line Feed Injection\n', ' 7: Cross-Site Scripting\n', ' 8: Template Injection\n', ' 9: SQL Injection\n', ' 10: Server-Side Request Forgery\n', ' 11: XML External Entity\n', ' 12: Remote Code Execution\n', ' 13: Memory Vulnerabilities\n', ' 14: Subdomain Takeover\n', ' 15: Race Conditions\n', ' 16: Insecure Direct Object References\n', ' 17: OAuth Vulnerabilities\n', ' 18: Application Logic and Conﬁguration Vulnerabilities\n', ' 19: Finding Your Own Bug Bounties\n', ' 20: Vulnerability Reports\nAppendix A: Tools\nAppendix B: Resources\nIndex\nCONTENTS IN DETAIL\nFOREWORD', 's when I was working on the early versions of this book. Jobert\nprovided an in-depth review, editing', ' to provide feedback\nand technical insights. His edits boosted my conﬁdence and taught me\nso much mo'

In [ ]:
# Create an empty list to hold the uppercase chapter titles
chapter_titles = []

# Loop through the list you created earlier (after_colon) and convert each item to uppercase
for item in after_colon:
    chapter_titles.append(item.upper())

# (Optional) inspect the result
print("Upper-cased chapter titles:\n", chapter_titles)


Upper-cased chapter titles:
 ['BUG BOUNTY BASICS', 'OPEN REDIRECT', 'HTTP PARAMETER POLLUTION', 'CROSS-SITE REQUEST FORGERY', 'HTML INJECTION AND CONTENT SPOOFING', 'CARRIAGE RETURN LINE FEED INJECTION', 'CROSS-SITE SCRIPTING', 'TEMPLATE INJECTION', 'SQL INJECTION', 'SERVER-SIDE REQUEST FORGERY', 'XML EXTERNAL ENTITY', 'REMOTE CODE EXECUTION', 'MEMORY VULNERABILITIES', 'SUBDOMAIN TAKEOVER', 'RACE CONDITIONS', 'INSECURE DIRECT OBJECT REFERENCES', 'OAUTH VULNERABILITIES', 'APPLICATION LOGIC AND CONFIGURATION VULNERABILITIES', 'FINDING YOUR OWN BUG BOUNTIES', 'RESOURCES\nINDEX\nCONTENTS IN DETAIL\nFOREWORD', 'S WHEN I WAS WORKING ON THE EARLY VERSIONS OF THIS BOOK. JOBERT\nPROVIDED AN IN-DEPTH REVIEW, EDITING', 'TO PROVIDE FEEDBACK\nAND TECHNICAL INSIGHTS. HIS EDITS BOOSTED MY CONFIDENCE AND TAUGHT ME\nSO MUCH MO', '19. HACKERONE HAS NEVER ASKED FOR ANYTHING IN RETURN. THEY\nONLY WANTED TO SUPPORT THE HACKING COMMU', '1. A DESCRIPTION OF THE VULNERABIL', 'YOU’RE INTERESTED IN, READ IT FIR

In [ ]:
chapter_titles = chapter_titles[0:19]

['BUG BOUNTY BASICS',
 'OPEN REDIRECT',
 'HTTP PARAMETER POLLUTION',
 'CROSS-SITE REQUEST FORGERY',
 'HTML INJECTION AND CONTENT SPOOFING',
 'CARRIAGE RETURN LINE FEED INJECTION',
 'CROSS-SITE SCRIPTING',
 'TEMPLATE INJECTION',
 'SQL INJECTION',
 'SERVER-SIDE REQUEST FORGERY',
 'XML EXTERNAL ENTITY',
 'REMOTE CODE EXECUTION',
 'MEMORY VULNERABILITIES',
 'SUBDOMAIN TAKEOVER',
 'RACE CONDITIONS',
 'INSECURE DIRECT OBJECT REFERENCES',
 'OAUTH VULNERABILITIES',
 'APPLICATION LOGIC AND CONFIGURATION VULNERABILITIES',
 'FINDING YOUR OWN BUG BOUNTIES']

In [ ]:
import pathlib, re, zipfile, textwrap
from PyPDF2 import PdfReader

### ------------------------------------------------------------------
### 1.  Get the book’s full text (reuse if you already have `full_text`)
### ------------------------------------------------------------------
try:
    full_text    # already in memory?
except NameError:
    reader     = PdfReader(pdf_name)
    full_text  = "\n".join(page.extract_text() or "" for page in reader.pages)

upper_text = full_text.upper()

### ------------------------------------------------------------------
### 2.  Find the SECOND occurrence of each chapter title
### ------------------------------------------------------------------
pos_list = []   # list of tuples: (title, second_hit_index)

for title in chapter_titles:
    # all occurrences (case-insensitive) in the entire book
    hits = [m.start() for m in re.finditer(re.escape(title), full_text)]
    if len(hits) < 2:
        print(f"⚠️  Title '{title}' found only {len(hits)} time(s); skipping.")
        continue
    pos_list.append((title, hits[1]))

# Sort by where they appear in the book
pos_list.sort(key=lambda t: t[1])

if not pos_list:
    raise RuntimeError("No chapters had a second occurrence—cannot split.")

### ------------------------------------------------------------------
### 3.  Slice text between successive start positions
### ------------------------------------------------------------------
OUT_DIR = pathlib.Path("/content/chapters_second")
OUT_DIR.mkdir(exist_ok=True)

def slug(s):   # safe filename
    return re.sub(r'[^0-9A-Za-z]+', '_', s).lower().strip('_')

for idx, (title, start) in enumerate(pos_list):
    end = pos_list[idx + 1][1] if idx + 1 < len(pos_list) else len(full_text)
    chunk = full_text[start:end].lstrip()        # trim leading whitespace

    fname = OUT_DIR / f"{slug(title)}.txt"
    fname.write_text(chunk, "utf-8")

    # quick confirmation
    preview = textwrap.shorten(chunk.replace("\n", " "), width=90, placeholder="…")
    print(f"✔️  {fname.name:35} | preview: {preview}")

### ------------------------------------------------------------------
### 4.  Zip the result
### ------------------------------------------------------------------
ZIP_PATH = "/content/chapters_second.zip"
with zipfile.ZipFile(ZIP_PATH, "w", zipfile.ZIP_DEFLATED) as zf:
    for p in OUT_DIR.rglob("*"):
        zf.write(p, arcname=p.relative_to(OUT_DIR))

print(f"\n✅ All done. Zipped chapters → {ZIP_PATH}")



⚠️  Title 'APPLICATION LOGIC AND CONFIGURATION VULNERABILITIES' found only 0 time(s); skipping.
⚠️  Title 'RESOURCES
INDEX
CONTENTS IN DETAIL
FOREWORD' found only 0 time(s); skipping.
⚠️  Title 'S WHEN I WAS WORKING ON THE EARLY VERSIONS OF THIS BOOK. JOBERT
PROVIDED AN IN-DEPTH REVIEW, EDITING' found only 0 time(s); skipping.
⚠️  Title 'TO PROVIDE FEEDBACK
AND TECHNICAL INSIGHTS. HIS EDITS BOOSTED MY CONFIDENCE AND TAUGHT ME
SO MUCH MO' found only 0 time(s); skipping.
⚠️  Title '19. HACKERONE HAS NEVER ASKED FOR ANYTHING IN RETURN. THEY
ONLY WANTED TO SUPPORT THE HACKING COMMU' found only 0 time(s); skipping.
⚠️  Title '1. A DESCRIPTION OF THE VULNERABIL' found only 0 time(s); skipping.
⚠️  Title 'YOU’RE INTERESTED IN, READ IT FIRST. IN SOME CASES, I
REFERENCE CONCEPTS DISCUSSED IN PREVIOUS' found only 0 time(s); skipping.
⚠️  Title 'S, BUT IN DOING SO, I TRY
TO NOTE WHERE I’VE DEFINED THE TERM SO YOU CAN REFER TO RELEVANT SECTIONS.' found only 0 time(s); skipping.
⚠️  Title 'BUG BOUN

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# (optional auto-download in Colab)
from google.colab import files
files.download(ZIP_PATH)
